# Live VCC image data -> distgen xy_dist file

See https://github.com/slaclab/lcls-lattice/blob/master/distgen/models/cu_inj/vcc_image/vcc_image.ipynb for a better explanation

In [1]:
import epics
import numpy as np
import os

In [2]:
BASEPV = 'CAMR:IN20:186'

SUFFIXES = {
    ':IMAGE': 'image array',
    ':N_OF_COL': 'start X',
    ':N_OF_ROW': 'Number of X pixels',
    ':RESOLUTION': 'size per pixel',
    ':RESOLUTION.EGU': 'units of size per pixel'
}
PVLIST = [BASEPV+s for s in SUFFIXES]
PVLIST

['CAMR:IN20:186:IMAGE',
 'CAMR:IN20:186:N_OF_COL',
 'CAMR:IN20:186:N_OF_ROW',
 'CAMR:IN20:186:RESOLUTION',
 'CAMR:IN20:186:RESOLUTION.EGU']

In [3]:
from skimage.filters import sobel
from skimage.segmentation import watershed

In [4]:
def isolate_image(img):
    """
    Uses a masking technique to isolate the VCC image
    """
    img=img.copy()
    elevation_map = sobel(img)
    markers = np.zeros_like(img)
    
    # TODO: tweak these numbers
    markers[img < .1] = 1
    markers[img > .2] = 2

    # Wateshed
    segmentation = watershed(elevation_map, markers)
    
    img[np.where(segmentation != 2)]  = 0 
    
    # 
    ixnonzero0 = np.nonzero(np.sum(img, axis=1))[0]
    ixnonzero1 = np.nonzero(np.sum(img, axis=0))[0]
    
    i0, i1, j0, j1 = ixnonzero0[0], ixnonzero0[-1], ixnonzero1[0], ixnonzero1[-1]
    cutimg = img[i0:i1,j0:j1]
    
    return cutimg

In [5]:
def write_distgen_xy_dist(filename, image, resolution, resolution_units='m'):
    """
    Writes image data in distgen's xy_dist format
    
    Returns the absolute path to the file written
    
    """
    
    # Get width of each dimension
    widths = resolution * np.array(image.shape)
    
    # Form header
    header = f"""x {widths[1]} {widths[1]/2} [{resolution_units}]
y {widths[0]} {widths[0]/2}  [{resolution_units}]"""
    
    # Save with the correct orientation
    np.savetxt(filename, np.flip(image, axis=0), header=header, comments='')
    
    return os.path.abspath(filename)

In [9]:
def get_live_distgen_xy_dist(filename='test.txt'):
    
    # Get
    dat = {k:v for k, v in zip(PVLIST, epics.caget_many(PVLIST))}
    
    arr = dat[BASEPV+':IMAGE']
    ncol = dat[BASEPV+':N_OF_COL']
    nrow = dat[BASEPV+':N_OF_ROW']
    resolution = dat[BASEPV+':RESOLUTION']
    resolution_units = dat[BASEPV+':RESOLUTION.EGU']
    image = arr.reshape(nrow, ncol)
        
    cutimg = isolate_image(image)
    
    fout = write_distgen_xy_dist(filename, cutimg, resolution, resolution_units=resolution_units)
    
    return fout
    
    
#get_live_distgen_xy_dist()    